In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

1.13.1
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.2
pandas 0.22.0
sklearn 0.19.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [2]:
import moxing as mox
mox.file.copy_parallel('s3://ai-awe-n4/aifood','./aifood/')
print("done")

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:pid: None.	1000/5001
INFO:root:pid: None.	2000/5001
INFO:root:pid: None.	3000/5001
INFO:root:pid: None.	4000/5001
INFO:root:pid: None.	5000/5001


done


In [3]:
images_dir = './aifood/images'
print(os.listdir(images_dir))

['三明治', '冰激凌', '土豆泥', '小米粥', '松鼠鱼', '烤冷面', '玉米饼', '甜甜圈', '芒果班戟', '鸡蛋布丁']


In [4]:
# 将图片转化成同一尺寸
height = 256
width = 256
channels = 3
batch_size = 64
num_classes = 10

# 读取图片并增强
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255, # 缩放到0-1之间
    rotation_range= 40, # 旋转范围
    width_shift_range= 0.2, # 水平位移
    height_shift_range= 0.2, # 竖直平移
    shear_range= 0.2, # 剪切范围
    zoom_range= 0.2, # 缩放范围
    horizontal_flip= True, # 随机水平翻转
    fill_mode= 'nearest', # 对空白位置的填充规则
)
# 按照文件夹进行分类，进行数据增强
train_generator = train_datagen.flow_from_directory(
    images_dir, target_size = (height, width), batch_size = batch_size,
    seed = 7, shuffle = True, class_mode = 'categorical')

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input)
valid_generator = valid_datagen.flow_from_directory(
    images_dir, target_size = (height, width), batch_size = batch_size,
    seed = 7, shuffle = False, class_mode = 'categorical')

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

Found 5000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
5000 5000


In [5]:
for i in range(1):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(64, 256, 256, 3) (64, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.

In [6]:
model = keras.models.Sequential()
model.add(keras.applications.ResNet50(
    include_top = False, pooling = 'avg', weights = 'imagenet'))
model.add(keras.layers.Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False

model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
output_model_file = './ai_food_model.h5'

# 定义回调函数
callbacks = [
#     keras.callbacks.TensorBoard(log_dir = logdir, profile_batch = 100000000),
    # 定义保存模型
    keras.callbacks.ModelCheckpoint(
        output_model_file, save_best_only = True, save_weights_only = False),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3),
]

epochs = 30
history = model.fit_generator(
    train_generator,
    steps_per_epoch= train_num // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps= valid_num // batch_size,
    callbacks = callbacks)

Epoch 1/30
79/79 [==============================] - 109s 1s/step - loss: 0.3269 - acc: 0.8946 - val_loss: 14.1298 - val_acc: 0.0938
Epoch 2/30
79/79 [==============================] - 97s 1s/step - loss: 0.3268 - acc: 0.8940 - val_loss: 14.3292 - val_acc: 0.1022
Epoch 3/30
79/79 [==============================] - 97s 1s/step - loss: 0.2996 - acc: 0.9024 - val_loss: 14.2860 - val_acc: 0.1010
Epoch 4/30
79/79 [==============================] - 96s 1s/step - loss: 0.2929 - acc: 0.9000 - val_loss: 14.5000 - val_acc: 0.0886
Epoch 5/30
79/79 [==============================] - 98s 1s/step - loss: 0.2805 - acc: 0.9094 - val_loss: 14.5337 - val_acc: 0.0988
Epoch 6/30
79/79 [==============================] - 97s 1s/step - loss: 0.2511 - acc: 0.9174 - val_loss: 14.4041 - val_acc: 0.1028


In [8]:
model.evaluate_generator(valid_generator,steps = valid_num // batch_size)

[13.479828154290967, 0.1106]

In [15]:
import moxing as mox
mox.file.copy('./ai_food_model.h5','s3://ai-awe-n4/tf_output1/model/resnet-50-tf1.h5')
print("done")

done


In [18]:
# 定义预测接口的inputs和outputs
# inputs和outputs字典的key值会作为模型输入输出tensor的索引键
# 模型输入输出定义需要和推理自定义脚本相匹配
predict_signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={"images" : model.input},
    outputs={"scores" : model.output}
)

# 定义保存路径
builder = tf.saved_model.builder.SavedModelBuilder('./tf_keras_saved_model/')



Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [20]:
with tf.Session(graph=tf.Graph()) as sess:
    builder.add_meta_graph_and_variables(

        sess,
        # 推理部署需要定义tf.saved_model.tag_constants.SERVING标签
        tags=[tf.saved_model.tag_constants.SERVING],
    #     """
    #     signature_def_map：items只能有一个，或者需要定义相应的key为
    #     tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
    #     """
        signature_def_map = {
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                predict_signature
        }

    )

    builder.save()
print('Done exporting!')

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: ./tf_keras_saved_model/saved_model.pb


INFO:tensorflow:SavedModel written to: ./tf_keras_saved_model/saved_model.pb


Done exporting!


In [21]:
!saved_model_cli show --dir ./tf_keras_saved_model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, -1, -1, 3)
        name: resnet50_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: dense/Softmax:0
  Method name is: tensorflow/serving/predict


In [24]:
import moxing as mox
mox.file.copy_parallel('./tf_keras_saved_model/','s3://ai-awe-n4/tf_output/tf_keras_saved_model')
print("done")

done
